##  Get shared link from datahub instructions

1. EGI Notebooks provides a ready to use access token that can be accessed from your notebooks and is automatically refreshed so you can always have a valid one.

The token is available at /var/run/secrets/egi.eu/access_token

2. Copy and paste the two functions below in your Jupyter Notebook or code.

3. You are READY to go. Retrieve EGI DataHub file shared links as in the example usage. 

## Documentation
https://docs.egi.eu/users/dev-env/notebooks/integration/#egi-services-access-tokens
https://docs.egi.eu/users/data/management/datahub/api/


## EGI Datahub functions to initialize EGI datahub and get shared link

In [7]:
def egi_datahub_init():
    import requests, json, datetime
    
    ONEZONE_ENDPOINT="https://datahub.egi.eu/api/v3/onezone/"
    ONEPROVIDER_ENDPOINT="https://cesnet-oneprovider-01.datahub.egi.eu/api/v3/oneprovider/"
    
    with open("/var/run/secrets/egi.eu/access_token") as f:
        EGI_TOKEN = f.read()
        try:
            # get DATAHUB token
            headers = {
                'X-Auth-Token': f"egi:" + EGI_TOKEN,
                'Content-type': 'application/json',
            }
            # get current timestamp
            ts = datetime.datetime.now().timestamp()
            data = json.dumps({ 
                'name': 'REST and CDMI access token ' + str(ts), 
                'type': { 
                    'accessToken': {} 
                }, 
                'caveats': [ { 
                    'type': 'interface', 
                    'interface': 'rest' 
                }] 
            })
            response = requests.post(ONEZONE_ENDPOINT+'user/tokens/named', headers=headers, data=data)
            print(json.dumps(response.json(), indent=2))
            DATAHUB_TOKEN=response.json()['token']              
            return DATAHUB_TOKEN
        except:
            print("EGI Datahub Authentication problem: check your credentials")

In [8]:
def egi_datahub_getlink(datahub_token, filename):
    import requests, json, os
    OIDC_ENDPOINT="https://aai.egi.eu/oidc/"
    ONEZONE_ENDPOINT="https://datahub.egi.eu/api/v3/onezone/"
    ONEPROVIDER_ENDPOINT="https://cesnet-oneprovider-01.datahub.egi.eu/api/v3/oneprovider/"
    
    bname = os.path.basename(filename)
    hname = filename.split('datahub/')[1]
    ## get file id
    headers = { 
        'X-Auth-Token': datahub_token 
    }
    response = requests.post(ONEPROVIDER_ENDPOINT+'lookup-file-id/'+hname, headers=headers)
    dh_fileid = response.json()['fileId']
    ## get shared link 
    headers = { 'X-Auth-Token': datahub_token, 'Content-Type': 'application/json',}
    data = json.dumps({ 
        'name': bname,
        'fileId': dh_fileid
    })
    response = requests.post(ONEPROVIDER_ENDPOINT+'shares', headers=headers, data=data)
    shareIdGenerated=response.json()['shareId']
    # get shared link details
    headers = {
        'X-Auth-Token': datahub_token
    }
    response = requests.get(ONEPROVIDER_ENDPOINT+'shares/'+shareIdGenerated, headers=headers)
    publicURL=response.json()['publicUrl'] #publicURL is nice link but to webpage where you can download file, not the actual file
    contentURL= ONEZONE_ENDPOINT+'shares/data/'+response.json()['rootFileId']+'/content' #use this link to aggregate Jupyter Notebooks
    return (publicURL, contentURL)

## example usage

### EGI DataHub initialization

In [9]:
DATAHUB_TOKEN = egi_datahub_init()

{
  "tokenId": "a11770898daf997e6b4221bbc732a2f3ch00e1",
  "token": "MDAxY2xvY2F00aW9uIGRhdGFodWIuZWdpLmV1CjAwNmJpZGVudGlmaWVyIDIvbm1kL3Vzci1mMDZmOWZmNDZjOWFjNGFhNjAyNDIwMmVmYmE5Mzc5MWNoOWFjNC9hY3QvYTExNzcwODk4ZGFmOTk3ZTZiNDIyMWJiYzczMmEyZjNjaDAwZTEKMDAxOWNpZCBpbnRlcmZhY2UgPSByZXN00CjAwMmZzaWduYXR1cmUgw658by6LPw00PBE2CRYh6z74yJxu1BJqBF01RfsPJaAMsK"
}


### get shared link to content

In [10]:
import os
volcano = 'Santorini2'
WORKDIR_FOLDER = os.path.join(os.environ['HOME'], "datahub/Reliance/Geohazard" + '_' + volcano)
INPUT_DATA_DIR  = os.path.join(WORKDIR_FOLDER, 'input')

myfilename = 'obs_sar.txt'
shared_res_path = os.path.join(INPUT_DATA_DIR, myfilename)
print(shared_res_path)
links = egi_datahub_getlink(DATAHUB_TOKEN, shared_res_path)
# we can aggregate either the publicURL or the contentURL as external resource in the research object
# the publicURL redirects to the EGI DataHub Webpage of the resource, where you can download the resource
# the contentURL redirects to the file contents directly. 
# NOTE: YOU NEED TO USE contentURL to aggregate JupyterNoteboks to be able to execute them from ROHub
res_file_url_public = links[0] 
print (res_file_url_public)
res_file_url_content = links[1] 
print (res_file_url_content)

/home/jovyan/datahub/Reliance/Geohazard_Santorini2/input/obs_sar.txt
https://datahub.egi.eu/share/e38fdf855d984239afda07c13ed5c769ch22a3
https://datahub.egi.eu/api/v3/onezone/shares/data/00000000007E1D76736861726547756964236665626130323533646361663965316432316135346465353737336237323766636838393662233732356634616233366362323664306662666330633132346337373565666565636865653439236533386664663835356439383432333961666461303763313365643563373639636832326133/content
